In [ ]:
import numpy as np

## This tests some hybridization 
### it is currently incorrect

In [ ]:
import numpy as np

# Convert A to numpy arrays
A = {
    'E': np.array([[1]]),
    'C3': np.array([[1]]),
    'C2': np.array([[1]]),
    'S4': np.array([[1]]),
    'sigma_d': np.array([[1]])
}

# Convert E to numpy arrays
E = {
    'E': np.array([[1, 0], [0, 1]]),
    'C3': np.array([[-1/2, -np.sqrt(3)/2], [np.sqrt(3)/2, -1/2]]),
    'C2': np.array([[1, 0], [0, -1]]),
    'S4': np.array([[-1/2, np.sqrt(3)/2], [-np.sqrt(3)/2, -1/2]]),
    'sigma_d': np.array([[-1, 0], [0, 1]])
}

# Initialize T2 dictionary with numpy arrays
T2 = {key: np.zeros((3, 3)) for key in A.keys()}

# Fill in the matrices for T2 using the representations from the image
T2['E'] = np.array([
    [1, 0, 0],
    [0, 1, 0],
    [0, 0, 1]
])

T2['C3'] = (1/3) * np.array([
    [-1, 2, 2],
    [2, -1, 2],
    [2, 2, -1]
])

T2['C2'] = np.array([
    [-1, 0, 0],
    [0, -1, 0],
    [0, 0, 1]
])

T2['S4'] = np.array([
    [0, -1, 0],
    [1, 0, 0],
    [0, 0, -1]
])

T2['sigma_d'] = np.array([
    [1, 0, 0],
    [0, -1, 0],
    [0, 0, 1]
])


In [ ]:
V = {
    'ss_sigma'      : -1.95933,
    's_star_s_star' : -4.24135,
    'ss_star_sigma' : -1.52230,
    'sp_sigma'      :  3.02562,
    's_star_p_sigma':  3.15565,
    'sd_sigma'      : -2.28485,
    's_star_d_sigma': -0.80993,
    'pp_sigma'      :  4.10364,
    'pp_pi'         : -1.51801,
    'pd_sigma'      : -1.35554,
    'pd_pi'         :  2.38479,
    'dd_sigma'      : -1.68136,
    'dd_pi'         :  2.58880,
    'dd_delta'      : -1.81400,
    
    'h_si_ss': -4.859509,
    'h_si_sp': 3.776178,
    'h_si_ss*': 0,
    'h_si_sd': -.007703,

    # atomic spin-orbit coupling (p-manifold)
    'SOC'           :  0.01989
}
E = {
    's'   : -2.15168,
    'px'  :  4.22925, 'py' : 4.22925, 'pz' : 4.22925,
    'dxy' : 13.78950, 'dyz': 13.78950, 'dzx': 13.78950,
    'dx2y2':13.78950, 'dz2': 13.78950,
    's*'  : 19.11650, 'HS' : -3.056510, 'delta_Si' : -0.276789, 'sp3': 20
}



In [11]:
import numpy as np
from numpy.linalg import svd

def make_hybrid_matrix(phi_deg=0.0):
    phi = np.deg2rad(phi_deg)
    c_s  = 0.5
    c_p  = np.sqrt(3)/2 * np.cos(phi)
    c_d  = 1.5 * np.sin(phi)

    # 4 bond directions, each /√3
    dirs = np.array([[ 1, 1, 1],
                     [ 1,-1,-1],
                     [-1, 1,-1],
                     [-1,-1, 1]], dtype=float)/np.sqrt(3)

    H = []
    for l, m, n in dirs:               # orbital order:  s, px, py, pz, dxy, dyz, dzx
        H.append([c_s,
                  c_p*l, c_p*m, c_p*n,
                  c_d*m*n, c_d*n*l, c_d*l*m])
    H = np.asarray(H)                  # shape (4, 7)

    # SVD‑based null space  → 3 residual SALCs
    u, s, vh = svd(H, full_matrices=True)
    rank = (s > 1e-12).sum()          
    bond_block   = vh[:rank].T         # (7, 4)
    leftover_blk = vh[rank:].T         # (7, 3)

    U = np.vstack([bond_block.T, leftover_blk.T])  # rows are |h₁⟩ … |h₇⟩
    return U                           # shape (7, 7), orthonormal

# quick self‑test
U = make_hybrid_matrix(phi_deg=0)      
print(U)

[[-5.00000000e-01 -5.00000000e-01 -5.00000000e-01 -5.00000000e-01
   0.00000000e+00  0.00000000e+00  0.00000000e+00]
 [-8.66025404e-01  2.88675135e-01  2.88675135e-01  2.88675135e-01
   0.00000000e+00  0.00000000e+00  0.00000000e+00]
 [-1.38777878e-17  5.77350269e-01 -7.88675135e-01  2.11324865e-01
   0.00000000e+00  0.00000000e+00  0.00000000e+00]
 [ 0.00000000e+00 -5.77350269e-01 -2.11324865e-01  7.88675135e-01
   0.00000000e+00  0.00000000e+00  0.00000000e+00]
 [ 0.00000000e+00  0.00000000e+00  0.00000000e+00  0.00000000e+00
   1.00000000e+00  0.00000000e+00  0.00000000e+00]
 [ 0.00000000e+00  0.00000000e+00  0.00000000e+00  0.00000000e+00
   0.00000000e+00  1.00000000e+00  0.00000000e+00]
 [ 0.00000000e+00  0.00000000e+00  0.00000000e+00  0.00000000e+00
   0.00000000e+00  0.00000000e+00  1.00000000e+00]]
